In [2]:
import os, re
import pandas as pd
import seaborn as sns
import yfinance as yf
import datetime 
import matplotlib.pyplot as plt
%matplotlib inline

### Parameters

### Dataset


#### Sentiment dataset

Trying to include all the sentiment dataset

https://stackoverflow.com/questions/4666973/how-to-extract-the-substring-between-two-markers

Example

In [55]:
rootdir = "../data"
regex = re.compile('(GE_*)')

filenames = []
colnames = []
for root, dirs, files in os.walk(rootdir):
  for file in files:
    # Extract file name
    if regex.match(file):
      filenames.append(file)
      
      # Extract dataset name
      found = re.search(r"(?<=GE_).*?(?=\.csv)", file).group(0)
      
      if found:
        phrases = found.split('_')
        if 'Llama' in phrases[1]:
            colname = f'{phrases[1]}-{phrases[-2]}-{phrases[-1]}' #, reg.group(1)
            
        else:
            colname = f'{phrases[1]}-{phrases[-1]}'
        colnames.append(colname)
        
colnames

['Llama-2-7B-GPTQ-fullarticles',
 'Llama-2-70B-GPTQ-headlines',
 'Llama-3-70B-GPTQ-fullarticles',
 'GPT3.5-headlines',
 'Llama-2-70B-GPTQ-fullarticles',
 'finBERT-headlines',
 'Llama-3-8B-GPTQ-fullarticles',
 'Llama-3-70B-GPTQ-headlines',
 'Llama-2-13B-GPTQ-fullarticles',
 'Llama-2-7B-GPTQ-headlines',
 'Llama-2-13b-GGUF-fullarticles',
 'Llama-3-8B-GPTQ-headlines',
 'Llama-2-13B-GPTQ-headlines']

In [56]:
data_list = []
for i in range(len(filenames)):
    dat = pd.read_csv(f'{rootdir}/{filenames[i]}', dtype={'Sentiment': float, 'Stock_symbol': str, 'Url': str}, parse_dates=['Date']).drop(['Stock_symbol', 'Url'], axis=1).set_index(['Date']).rename(columns={"Sentiment": colnames[i]})
    data_list.append(dat)
    
merged_df = pd.concat(data_list, axis=1)
merged_df

,Llama-2-7B-GPTQ-fullarticles,Llama-2-70B-GPTQ-headlines,Llama-3-70B-GPTQ-fullarticles,GPT3.5-headlines,Llama-2-70B-GPTQ-fullarticles,finBERT-headlines,Llama-3-8B-GPTQ-fullarticles,Llama-3-70B-GPTQ-headlines,Llama-2-13B-GPTQ-fullarticles,Llama-2-7B-GPTQ-headlines,Llama-2-13b-GGUF-fullarticles,Llama-3-8B-GPTQ-headlines,Llama-2-13B-GPTQ-headlines
Date,,,,,,,,,,,,,
2023-12-16 21:00:00+00:00,3.0,3.0,4.0,4.0,4.0,4.856880,4.0,3.0,2.0,4.0,4.0,4.0,3.0
2023-12-16 00:00:00+00:00,4.0,4.0,5.0,5.0,4.0,4.722641,4.0,5.0,4.0,3.0,4.0,4.0,3.0
2023-12-16 00:00:00+00:00,3.0,3.0,4.0,3.0,4.0,3.151579,3.0,3.0,4.0,3.0,4.0,3.0,3.0
2023-12-16 00:00:00+00:00,3.0,4.0,4.0,4.0,2.0,4.663424,0.0,3.0,3.0,4.0,4.0,3.0,3.0
2023-12-15 00:00:00+00:00,4.0,3.0,4.0,3.0,4.0,2.593138,3.0,3.0,3.0,3.0,4.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-04-23 00:00:00+00:00,4.0,4.0,4.0,5.0,4.0,3.744149,4.0,5.0,3.0,4.0,3.0,4.0,4.0
2012-04-23 00:00:00+00:00,3.0,3.0,4.0,3.0,3.0,2.466803,3.0,3.0,4.0,3.0,4.0,3.0,3.0
2012-04-20 00:00:00+00:00,4.0,4.0,5.0,4.0,4.0,3.857823,4.0,4.0,4.0,4.0,4.0,3.0,3.0


There are no missing values

In [57]:
merged_df.isna().sum()

Llama-2-7B-GPTQ-fullarticles     0
Llama-2-70B-GPTQ-headlines       0
Llama-3-70B-GPTQ-fullarticles    0
GPT3.5-headlines                 0
Llama-2-70B-GPTQ-fullarticles    0
finBERT-headlines                0
Llama-3-8B-GPTQ-fullarticles     0
Llama-3-70B-GPTQ-headlines       0
Llama-2-13B-GPTQ-fullarticles    0
Llama-2-7B-GPTQ-headlines        0
Llama-2-13b-GGUF-fullarticles    0
Llama-3-8B-GPTQ-headlines        0
Llama-2-13B-GPTQ-headlines       0
dtype: int64

Compare mean and range of values among the results

In [58]:
merged_df.describe()

,Llama-2-7B-GPTQ-fullarticles,Llama-2-70B-GPTQ-headlines,Llama-3-70B-GPTQ-fullarticles,GPT3.5-headlines,Llama-2-70B-GPTQ-fullarticles,finBERT-headlines,Llama-3-8B-GPTQ-fullarticles,Llama-3-70B-GPTQ-headlines,Llama-2-13B-GPTQ-fullarticles,Llama-2-7B-GPTQ-headlines,Llama-2-13b-GGUF-fullarticles,Llama-3-8B-GPTQ-headlines,Llama-2-13B-GPTQ-headlines
count,8680.000000,8680.000000,8680.000000,8680.000000,8680.000000,8680.000000,8680.000000,8680.000000,8680.000000,8680.000000,8680.000000,8680.000000,8680.000000
mean,3.245622,3.066590,3.268203,3.331452,2.999654,2.614096,2.778571,3.217281,3.181452,3.258065,3.308525,2.711982,2.997811
std,0.636879,0.904057,1.217533,1.053660,1.040542,1.147998,1.219556,1.008445,0.709361,0.458175,0.721964,1.012763,0.408877
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.078855,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000
25%,3.000000,3.000000,2.000000,3.000000,3.000000,2.445793,2.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
50%,3.000000,3.000000,4.000000,3.000000,3.000000,2.560233,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
75%,4.000000,4.000000,4.000000,4.000000,4.000000,2.954813,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,3.000000
max,5.000000,5.000000,5.000000,5.000000,7.000000,4.857064,50.000000,5.000000,7.000000,5.000000,7.000000,5.000000,5.000000


Check value distribution

In [59]:
for column in merged_df.columns: #[:-1]
    print(merged_df[column].value_counts())

Llama-2-7B-GPTQ-fullarticles
3.0    5440
4.0    2378
2.0     594
5.0     221
1.0      47
Name: count, dtype: int64
Llama-2-70B-GPTQ-headlines
3.0    4812
4.0    2569
1.0     989
2.0     211
5.0      99
Name: count, dtype: int64
Llama-3-70B-GPTQ-fullarticles
4.0    3614
3.0    1728
2.0    1287
5.0    1045
1.0     929
0.0      77
Name: count, dtype: int64
GPT3.5-headlines
3.0    3089
4.0    2813
5.0    1127
2.0    1112
1.0     539
Name: count, dtype: int64
Llama-2-70B-GPTQ-fullarticles
3.0    3383
4.0    2998
1.0    1182
2.0     948
5.0     161
0.0       6
7.0       1
6.0       1
Name: count, dtype: int64
finBERT-headlines
2.491283    31
2.591554    24
2.367193    19
2.593138    16
2.648294    16
            ..
2.534693     1
2.522934     1
2.982258     1
2.488833     1
2.524730     1
Name: count, Length: 8105, dtype: int64
Llama-3-8B-GPTQ-fullarticles
3.0     3597
4.0     2292
2.0     1288
1.0     1276
0.0      185
5.0       32
10.0       7
50.0       1
7.0        1
20.0       1
Name: c

Time stamps of sentiment data are in GMT+0 timezone

In [60]:
merged_df.index[0]

Timestamp('2023-12-16 21:00:00+0000', tz='UTC')

In [61]:
def date_extract(timestamp):
    match_str = re.search(r'\d{4}-\d{2}-\d{2}', str(timestamp))
    res = datetime.datetime.strptime(match_str.group(), '%Y-%m-%d').date()
    return res
    

In [62]:
# match_str = [re.search(r'\d{4}-\d{2}-\d{2}', str(x)) for x in merged_df.index]
# res = [datetime.datetime.strptime(x.group(), '%Y-%m-%d').date() for x in match_str]
merged_df['Time'] = [date_extract(x) for x in merged_df.index]
merged_df.head()

,Llama-2-7B-GPTQ-fullarticles,Llama-2-70B-GPTQ-headlines,Llama-3-70B-GPTQ-fullarticles,GPT3.5-headlines,Llama-2-70B-GPTQ-fullarticles,finBERT-headlines,Llama-3-8B-GPTQ-fullarticles,Llama-3-70B-GPTQ-headlines,Llama-2-13B-GPTQ-fullarticles,Llama-2-7B-GPTQ-headlines,Llama-2-13b-GGUF-fullarticles,Llama-3-8B-GPTQ-headlines,Llama-2-13B-GPTQ-headlines,Time
Date,,,,,,,,,,,,,,
2023-12-16 21:00:00+00:00,3.0,3.0,4.0,4.0,4.0,4.856880,4.0,3.0,2.0,4.0,4.0,4.0,3.0,2023-12-16
2023-12-16 00:00:00+00:00,4.0,4.0,5.0,5.0,4.0,4.722641,4.0,5.0,4.0,3.0,4.0,4.0,3.0,2023-12-16
2023-12-16 00:00:00+00:00,3.0,3.0,4.0,3.0,4.0,3.151579,3.0,3.0,4.0,3.0,4.0,3.0,3.0,2023-12-16
2023-12-16 00:00:00+00:00,3.0,4.0,4.0,4.0,2.0,4.663424,0.0,3.0,3.0,4.0,4.0,3.0,3.0,2023-12-16
2023-12-15 00:00:00+00:00,4.0,3.0,4.0,3.0,4.0,2.593138,3.0,3.0,3.0,3.0,4.0,3.0,3.0,2023-12-15


Reduce sentiment score by day.

First attempt: simple average.

In [63]:
sent_daily = merged_df.groupby(['Time']).mean()
sent_daily

,Llama-2-7B-GPTQ-fullarticles,Llama-2-70B-GPTQ-headlines,Llama-3-70B-GPTQ-fullarticles,GPT3.5-headlines,Llama-2-70B-GPTQ-fullarticles,finBERT-headlines,Llama-3-8B-GPTQ-fullarticles,Llama-3-70B-GPTQ-headlines,Llama-2-13B-GPTQ-fullarticles,Llama-2-7B-GPTQ-headlines,Llama-2-13b-GGUF-fullarticles,Llama-3-8B-GPTQ-headlines,Llama-2-13B-GPTQ-headlines
Time,,,,,,,,,,,,,
2012-04-20,3.666667,3.333333,4.666667,3.333333,3.333333,2.935755,3.666667,3.333333,3.666667,4.000000,3.666667,3.000000,3.000000
2012-04-23,3.333333,3.333333,3.666667,3.666667,3.333333,2.869610,3.333333,3.666667,3.333333,3.333333,3.333333,3.333333,3.333333
2012-04-24,3.000000,4.000000,3.000000,4.000000,3.000000,2.634394,3.000000,3.000000,4.000000,3.000000,4.000000,4.000000,3.000000
2012-04-26,3.000000,3.000000,3.000000,3.000000,3.000000,0.777762,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
2012-04-30,3.000000,3.000000,3.000000,3.000000,4.000000,2.604599,3.000000,3.000000,4.000000,3.000000,4.000000,3.000000,3.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-11,3.000000,2.800000,3.200000,2.800000,3.200000,2.441272,2.400000,2.600000,2.800000,3.400000,3.200000,2.200000,2.800000
2023-12-12,3.750000,3.375000,4.125000,3.750000,3.500000,2.958181,3.375000,3.500000,3.750000,3.250000,3.750000,3.250000,3.125000
2023-12-13,3.833333,3.666667,3.666667,4.000000,3.666667,3.125993,3.166667,4.000000,3.500000,3.500000,3.666667,3.500000,3.000000


#### Stock price data

In [71]:
start_date = sent_daily.index[0]
end_date = sent_daily.index[-1]

stock_price = yf.Ticker('GE').history(start=start_date, end=end_date, period="1day", auto_adjust=False, actions=False)[['Adj Close']]

Time stamps of GE are in GMT-4/-5 timezone

In [72]:
stock_price.index[0]

Timestamp('2012-04-20 00:00:00-0400', tz='America/New_York')

In [73]:
stock_price['Time'] = [date_extract(x) for x in stock_price.index]
print('Number of duplication in time: ', sum(stock_price['Time'].value_counts() >1))

Number of duplication in time:  0


In [74]:
stock_price.index = stock_price['Time']
stock_price.head()

,Adj Close,Time
Time,,
2012-04-20,68.549644,2012-04-20
2012-04-23,67.522812,2012-04-23
2012-04-24,69.186989,2012-04-24
2012-04-25,68.868317,2012-04-25
2012-04-26,69.470230,2012-04-26


#### Fama-French 3 factors

https://sec-api.io/resources/fama-french-factor-model

Retrieve FF3 so that timeframe matches sentiment data 

In [75]:
print(start_date, end_date)

2012-04-20 2023-12-16


In [76]:
ff3 = pd.read_csv('../data/F-F_Research_Data_Factors_daily.csv', index_col=0).loc['20120420':'20231101']
ff3

,Mkt-RF,SMB,HML,RF
20120420,0.11,0.54,-0.10,0.000
20120423,-0.95,-0.56,0.15,0.000
20120424,0.29,0.26,0.91,0.000
20120425,1.44,0.40,-0.98,0.000
20120426,0.75,-0.02,-0.06,0.000
...,...,...,...,...
20231026,-1.15,0.85,1.66,0.021
20231027,-0.53,-0.35,-0.57,0.021
20231030,1.15,-0.39,0.28,0.021
20231031,0.63,0.05,-0.08,0.021


Convert 'ff3.index' so that it has the same format with sentiment data

In [77]:
new_ind = [datetime.datetime.strptime(x, '%Y%m%d').date() for x in ff3.index]
ff3.index = new_ind
ff3.index.name = 'Time'
ff3.head()

,Mkt-RF,SMB,HML,RF
Time,,,,
2012-04-20,0.11,0.54,-0.10,0.0
2012-04-23,-0.95,-0.56,0.15,0.0
2012-04-24,0.29,0.26,0.91,0.0
2012-04-25,1.44,0.40,-0.98,0.0
2012-04-26,0.75,-0.02,-0.06,0.0


#### Final full dataset
Combine sentiment data with factors and stock price

In [78]:
print(type(sent_daily.index[0]))
print(type(ff3.index[0]))
print(type(stock_price.index[0]))

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>


In [79]:
df_full = pd.concat([sent_daily, ff3, stock_price],
                    axis=1, 
                    join='inner')
df_full

,Llama-2-7B-GPTQ-fullarticles,Llama-2-70B-GPTQ-headlines,Llama-3-70B-GPTQ-fullarticles,GPT3.5-headlines,Llama-2-70B-GPTQ-fullarticles,finBERT-headlines,Llama-3-8B-GPTQ-fullarticles,Llama-3-70B-GPTQ-headlines,Llama-2-13B-GPTQ-fullarticles,Llama-2-7B-GPTQ-headlines,Llama-2-13b-GGUF-fullarticles,Llama-3-8B-GPTQ-headlines,Llama-2-13B-GPTQ-headlines,Mkt-RF,SMB,HML,RF,Adj Close,Time
Time,,,,,,,,,,,,,,,,,,,
2012-04-20,3.666667,3.333333,4.666667,3.333333,3.333333,2.935755,3.666667,3.333333,3.666667,4.000000,3.666667,3.000000,3.000000,0.11,0.54,-0.10,0.000,68.549644,2012-04-20
2012-04-23,3.333333,3.333333,3.666667,3.666667,3.333333,2.869610,3.333333,3.666667,3.333333,3.333333,3.333333,3.333333,3.333333,-0.95,-0.56,0.15,0.000,67.522812,2012-04-23
2012-04-24,3.000000,4.000000,3.000000,4.000000,3.000000,2.634394,3.000000,3.000000,4.000000,3.000000,4.000000,4.000000,3.000000,0.29,0.26,0.91,0.000,69.186989,2012-04-24
2012-04-26,3.000000,3.000000,3.000000,3.000000,3.000000,0.777762,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,0.75,-0.02,-0.06,0.000,69.470230,2012-04-26
2012-04-30,3.000000,3.000000,3.000000,3.000000,4.000000,2.604599,3.000000,3.000000,4.000000,3.000000,4.000000,3.000000,3.000000,-0.49,-0.57,0.00,0.000,69.328606,2012-04-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-26,3.600000,3.600000,4.200000,4.000000,3.800000,2.274692,3.400000,4.000000,3.400000,3.800000,3.800000,3.200000,3.000000,-1.15,0.85,1.66,0.021,86.609253,2023-10-26
2023-10-27,3.666667,3.333333,4.333333,3.000000,4.000000,1.883527,3.333333,3.000000,3.666667,3.000000,4.000000,2.333333,3.000000,-0.53,-0.35,-0.57,0.021,84.658951,2023-10-27
2023-10-30,2.000000,1.000000,4.000000,2.000000,4.000000,2.473848,3.000000,2.000000,4.000000,3.000000,2.000000,2.000000,2.000000,1.15,-0.39,0.28,0.021,87.413254,2023-10-30


In [81]:
# df_full.to_csv('../data/df_full.csv')

#### Train-test split with purge

### Model